# Trabalho 3: Avaliando sistemas de IR

Neste trabalho, o aluno irá explorar o problema de otimização de sistemas de IR através da análise de métricas de avaliação populares para esse tipo de sistema.

Para isso, vamos usar a base CFC (com artigos sobre fibrose cística), a qual encontra-se nesta pasta.

Observação: a idéia aqui é usar o jupyter para mostrar a evolução dos teus experimentos. Então use ele adequadamente. Ou seja, não altere funções do começo para reexecutar algo que foi feito lá embaixo. Neste caso, sobreescreva a função lá embaixo. A leitura do notebook tem que ser feita sequencialmente para ficar fácil, ok?


## Passo 1 - Normalização

Primeira coisa que você vai precisar fazer é ler os arquivos do CFC. Essa é uma parte meio braçal mesmo, mas não tem como escapar dela. Então você terá que criar funções que consigam ler cada arquivo do CFC e parsear eles.


In [35]:
import pandas as pd


def printJson(data):
    print(json.dumps(data, indent=4))

def extractCFCArticles(path):
    # Lista de campos do arquivo CFC
    fields = [
            'PN',
            'RN',
            'AN',
            'AU',
            'TI',
            'SO',
            'MJ',
            'MN',
            'AB',
            'EX',
            'RF',
            'CT',
    ]
    
    # Lista de campos que mudarao seus nomes
    cast = {
        'RN': 'record',
        'AB': 'content',
        'EX': 'content',
        'AN': 'an',
        'TI': 'title'
    }
    
    file = open(path).read()
    articles = []

    for article in file.split("\n\n"):
        # Objeto do artigo
        articleData = {
            'title':'',
            'record': 0,
            'year': 0,
            'number': 0,
            'an': 0,
            'AU': '',
            'SO': '',
            'MJ': [],
            'MN': [],
            'RF': [],
            'CT': [],
            'contentType': '',
            'content': '',
        }
        currentField = ''
        for line in article.split("\n"):
            # Linha vazia ou fim de arquivo
            if(not line.strip() or line.startswith("\x1a")):
                continue 
            
            for key in fields:
                # Verificase a linha é a definição de um novo atributo
                if(line.startswith(key + '')):
                    currentField = line[0:2].upper()
                    line = line[3:]
            line = line.strip()
            if(currentField == 'RF' or currentField == 'CT'):
                articleData[currentField].append(line)
                continue
                
            if(currentField == 'MJ' or currentField == 'MN'):
                articleData[currentField]+= line.split("  ")
                continue
            
            if(currentField == 'PN'):
                articleData['year'] = int(line[0:2])
                articleData['number'] = int(line[2])
                continue
            
            if(currentField == 'AB' or currentField == 'EX'):
                articleData['contentType'] = currentField
                
            if(currentField in cast):
                currentField = cast[currentField]
            
            if(currentField == 'record' or currentField == 'an'):
                articleData[currentField] = int(line)
                continue
                
                
            if(articleData[currentField]):
                articleData[currentField]+=' '
            articleData[currentField]+= line 
        
        if(articleData['title']):
            articles.append(articleData)
    return articles
            

articles = [
    'data/cf74',
    'data/cf75',
    'data/cf76',
    'data/cf77',
    'data/cf78',
    'data/cf79'
]

import json
data = extractCFCArticles(articles[5])

# print(json.dumps(data[0], indent=4))

# pd.DataFrame(data).to_json(orient='table')
# for i in data:
#     print(i['AB'] and i['EX'])
#     print(i, "\n--------------------------------------------------\n")

## Passo 2 - Primeira indexação

Depois de parsear os arquivos, agora é hora de indexar os dados. Ao invés de criar o código do zero, vamos usar o Whoosh, que é uma implementação em python inspirada no Lucene. A documentação pode ser encontrada em https://whoosh.readthedocs.io/en/latest/index.html.

Nesta questão de indexação, temos que fazer algumas escolhas: 
* Como fazer o processo de tokenização? 
* Stemmizar ou não stemmizar, eis a questão...
* Quais campos são úteis para indexação?

Neste momento, você vai tomar suas decisões iniciais. A ideia é você testar, depois, outras soluções para verificar quais tiveram os melhores resultados, entendeu? Então não esqueça de documentar, aqui, qual a sua decisão inicial e depois ir explicando ao longo do notebook os experimentos que está fazendo.

Para pesquisa e indexação, iremos avaliar principalmente os subjects. Estes, levantados por especialistas, contém informações sobre palavras-chave que definem os artigos. 

No geral, cada artigo possui um Major e Minor Subject. O primeiro define o foco do artigo enquanto o segundo, temas secundários abordados.

Dentro dos subjects, os termos são definidos seguindo o [vocabulário médico MeSH](https://meshb.nlm.nih.gov/), em alguns casos seguido de uma sigla de qualificadores (Ex di:diagnóstico).

A decisão inicial é usar destas informações para gerar indexação e ranqueamento de pesquisas. Priorizando o Major subject junto do qualificador, em seguida minor subjects

Usaremos também os autores e citações para calcular o H-index dos artigos

In [52]:
from os import mkdir
from shutil import rmtree
from whoosh.index import create_in
from whoosh.fields import *

db_name = "indexdir"


schema = Schema(title=TEXT(stored=True), 
                record=NUMERIC(stored=True), 
                year=NUMERIC(stored=True),
                number=NUMERIC(stored=True),
                an=NUMERIC(stored=True),
                content=TEXT)

rmtree(db_name)
os.mkdir(db_name)
ix = create_in(db_name, schema)
writer = ix.writer()

for file in articles[1:]:
    for article in extractCFCArticles(articles[1]):
    #     print(json.dumps(article, indent=4))

        writer.add_document(title=article['title'],
                            record=article['record'],
                            year=article['year'],
                            number=article['number'],
                            an=article['an'],
                            content=article['content'],
                           )

writer.commit()



In [63]:
from whoosh.qparser import QueryParser

with ix.searcher() as searcher:
    print(list(searcher.lexicon("content")))
         
    query = QueryParser("content", ix.schema).parse("What are the effects of calcium on the physical properties of mucus from CF patients?")
    results = searcher.search(query)
#     print(len(results), query,results)

[b'0.0001', b'0.0005', b'0.001', b'0.002', b'0.005', b'0.01', b'0.02', b'0.025', b'0.08', b'0.1', b'0.3', b'0.32', b'0.4', b'0.49', b'0.50', b'0.56', b'0.6', b'0.70', b'0.75', b'0.934', b'000', b'0700', b'0ronchial', b'1.0', b'1.03', b'1.2', b'1.43', b'1.5', b'1.6', b'1.6.4.2', b'1.73', b'1.79', b'10', b'10.61', b'100', b'1000', b'104', b'105', b'1054', b'1055', b'107', b'11', b'11.2', b'11.9', b'111', b'112', b'113.7', b'11s', b'12', b'12.7', b'129', b'13', b'13.47', b'130', b'131i', b'135', b'14', b'14.01', b'144', b'145.2', b'146', b'14c', b'15', b'150', b'151', b'16', b'16.7', b'163', b'1651', b'169', b'17', b'173', b'176', b'18', b'1825', b'19', b'1930', b'1939', b'1940', b'1953', b'1954', b'1962', b'1964', b'1966', b'1967', b'1968', b'1969', b'1971', b'1972', b'1973', b'1974', b'1975', b'2.0', b'2.1', b'2.3', b'2.5', b'2.6', b'2.86', b'20', b'20.1', b'200', b'202', b'203.2', b'21', b'2100', b'22', b'229', b'23', b'2360', b'24', b'25', b'26', b'27', b'28', b'282', b'29', b'3.0', b

## Passo 3 - Calculando as métricas de avaliação

Para avaliar a tua decisão, vamos usar como métricas a precisão, recall e f-measure. Faça uma variação da precisão e recall como P@n e R@n, pois serão úteis para os gráficos que serão gerados. 

In [ ]:
#códigos...

## Passo 4 - Gráficos do experimento 1

Apresentar agora os gráficos do teu experimento. Você deve gerar um gráfico PxR, como visto em sala. Para gerar gráficos, você pode usar o matplotlib. Se quiser usar outra biblioteca que ache mais fácil, sem problemas!

In [ ]:
# códigos...

## Passo 5 - Demais experimentos

Agora você irá testar novas configurações de campos e configurações para tentar encontrar qual a que te dá melhores resultados. Mostrar, como um relatório, a evolução do trabalho. Ou seja, mostrar como o trabalho foi evoluindo para você alcançar o melhor resultado (quais modificações foram feitas, como cada modificação influenciou nas métricas, etc).

In [ ]:
# códigos...

## Passo 6 - Quantos resultados eu devo voltar para o usuário?

Um dos problemas de sistemas de IR é determinar quantos resultados você deve retornar ao usuário. Uma forma de entender o comportamento do sistema e qual o corte ideal na lista resultante é através do uso de curvas ROC. Assim, plote a curva ROC do teu sistema e determine, através da análise da curva, qual o ponto ideal de corte para o teu sistema.

In [ ]:
# códigos...